<a href="https://colab.research.google.com/github/kapibara3chi/NLPTesForGC/blob/main/NLPTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [1]:
!pip  install -U ginza ja_ginza_electra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 10.2 MB/s 
     |████████████████████████████████| 2.2 MB 8.9 MB/s 
     |████████████████████████████████| 283 kB 45.6 MB/s 
     |████████████████████████████████| 53 kB 1.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.2 MB/s 
     |████████████████████████████████| 4.7 MB 40.3 MB/s 
     |████████████████████████████████| 7.6 MB 31.2 MB/s 
     |████████████████████████████████| 182 kB 57.1 MB/s 
     |████████████████████████████████| 6.6 MB 40.4 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=2be98ca54665e63dbad3a0aaeaf0b899581b2c319717fb996f3084c7387da92c
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
  Created wheel for sudachitra: filename=SudachiTra-0.1.7-py3-none-any.whl size=266095 sha256=51fde

# main

---

## algol
- 名詞の親をたどっていく
- 過程の動詞を抽出
- 名詞にたどり着くまで探す

### 関係性の抽出
- 名詞(NOUN,PROPN)→ROOTまでたどる
- VERB →  NOUNで名詞の状態がわかる
- 矢印を遡る方向に関連付ける=head

#### 要件要件
- 単語間の関係性関係性
- 単語の単語に与える影響（プラス、マイナス）
- 単語のDB化
- 単語のカテゴライズ
  - 予め作成作成
- 動詞のネガポジ
  - 予め作成 or ネットの辞書利用

- Extract noun-verb pairs
- Extract compound nouns
- 単語DBを自動生成
- 動詞と動詞での関連もあり ex) 分離改善するが、コンタミ悪化※トレードオフ、相関どう拾う？ → 名詞- 名詞の関係性に変換する
- 単語と課題、特性値、パラメータは後でカテゴリ分けする → DB作成
- 動詞のネガポジ判定 → 自作する？
  - あご → 出す/下げる＝p
  - 改善する=p



### 名詞の抽出
compoundで複合語をまとめる

###  条件の抽出
caseで判定する判定する

### 名詞  → 動詞 
- obj  ex) あご → 出す

### 名詞  → 名詞
- coumpound  ex) 下流 → あご
- nummod ex)2 → mm

### 動詞 → 動詞
- advcl  ex) だす → 改善

|課題| element元(課題/特性値/パラメータ)|状態|element先|状態|case|Doc link| 
|--|--| -- | --| -- | --|--|
|分離|分離R|悪化|下流あご|出す|s|s|

※要件要素 → 課題‥の分類は後で行う

|課題  |Doc| 
| -- | --|
|分離|httb:|

|単語  | 状態|関係性|Doc|
| -- | --|-- |--|
|あごあご|出す出す|i|http:|



In [2]:
import ginza
from spacy import displacy
import spacy
class DependencyAnalysis:
    def __init__(self):
        self.nlp = spacy.load('ja_ginza_electra')
        self.indices = []

    def get_analysis(self, text):
        """係り受け解析"""
        doc = self.nlp(text)

        token_head_list = []

        for sent in doc.sents:
            for token in sent:
                token_head_list.append(
                    {"i": token.i, "orth": token.orth_, "base": token.lemma_,
                     "head": token.head.i, "dep": token.dep_,"pos":token.pos_,
                     "tag":token.tag_})

        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

        return token_head_list
    # def get(self, text, target):
    #   # 係り受け解析
    #   self.depend_indices = self.get_analysis(text)
    #   print(self.depend_indices)

    def get_words(self,  m, head):
        """対象の領域の抽出"""
        
        if m["head"] == head :
            for n in self.depend_indices:
                self.get_words(m=n, head=m["i"])
                            
            self.indices.append(m["i"])
        
        return
    def get(self, text, target):
        
        # 係り受け解析
        self.depend_indices = self.get_analysis(text)
        # print(self.depend_indices)
        for i in self.depend_indices:
          print(i)
                
        # 係り元のインデックスを取得
        head = self.depend_indices[target]["head"]
        # print(head)

        # 係り元のかかり先を探索
        self.indices.append(head)
        print(self.indices)#list of head?
        for m in self.depend_indices:
            if m["head"] == head and m["dep"] != "ROOT":
                self.get_words(m, head=head)

        clause = ""
        for clause_num in sorted(self.indices):
            clause = clause + str(self.depend_indices[clause_num]["orth"])
            
        return clause

if __name__ == '__main__':
  dependency = DependencyAnalysis()
  txt="下流あごを2mmだすと分離は改善するが、コンタミが悪化する。 非通紙部昇温時のxx75において発生する。"
  # ret=dependency.get_analysis("下流あごをだすと分離は改善するが、コンタミが悪化する")
  # for i in ret:
    # print(i)
  ret=dependency.get(txt,0)
  print(ret)


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
error megagonlabs/transformers-ud-japanese-electra-base-ginza-510 does not appear to have a file named config.json.
error We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like /tmp/tmpstfa5j38/config.json is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.
trying to download model from huggingface hub: megagonlabs/transformers-ud-japanese-electra-base-ginza-510 ...


succeded


{'i': 0, 'orth': '下流', 'base': '下流', 'head': 1, 'dep': 'compound', 'pos': 'NOUN', 'tag': '名詞-普通名詞-一般'}
{'i': 1, 'orth': 'あご', 'base': 'あご', 'head': 5, 'dep': 'obj', 'pos': 'NOUN', 'tag': '名詞-普通名詞-一般'}
{'i': 2, 'orth': 'を', 'base': 'を', 'head': 1, 'dep': 'case', 'pos': 'ADP', 'tag': '助詞-格助詞'}
{'i': 3, 'orth': '2', 'base': '2', 'head': 4, 'dep': 'nummod', 'pos': 'NUM', 'tag': '名詞-数詞'}
{'i': 4, 'orth': 'mm', 'base': 'mm', 'head': 5, 'dep': 'obl', 'pos': 'NOUN', 'tag': '名詞-普通名詞-助数詞可能'}
{'i': 5, 'orth': 'だす', 'base': 'だす', 'head': 9, 'dep': 'advcl', 'pos': 'VERB', 'tag': '動詞-非自立可能'}
{'i': 6, 'orth': 'と', 'base': 'と', 'head': 5, 'dep': 'mark', 'pos': 'SCONJ', 'tag': '助詞-接続助詞'}
{'i': 7, 'orth': '分離', 'base': '分離', 'head': 9, 'dep': 'nsubj', 'pos': 'NOUN', 'tag': '名詞-普通名詞-サ変可能'}
{'i': 8, 'orth': 'は', 'base': 'は', 'head': 7, 'dep': 'case', 'pos': 'ADP', 'tag': '助詞-係助詞'}
{'i': 9, 'orth': '改善', 'base': '改善', 'head': 15, 'dep': 'advcl', 'pos': 'VERB', 'tag': '名詞-普通名詞-サ変可能'}
{'i': 10, 'orth': 'する',

## Universal POS Tags version2
[URL](https://www.jstage.jst.go.jp/article/jnlp/26/1/26_3/_pdf/-char/ja)

|Tag  | 品詞| 
| -- | --|
ADJ|形容詞
ADV|副詞
VERB|動詞
NOUN|名詞
PROPN|固有名詞
NUM|数詞

### 節レベルの依存関係
主語や目的語など、述語の項や修飾語を表す依存関係

|Label| 依存関係| 
| -- | --|
obj|目的語
iobj|間接目的語
advcl|副詞的修飾節
nsubj|名詞主語

### 名詞句レベルの依存関係
名詞句を構成する修飾語や機能語を表す依存関係

|Label| 依存関係| 
| -- | --|
nummod|数詞
case|各標識
acl|形容詞的修飾節

### その他の依存関係
それ以外のもの

|Label| 依存関係| 
| -- | --|
compound|複合語